https://www.kaggle.com/code/imsparsh/food-classifier-mobilenetv2-resnet50-vgg16

In [1]:
# importing libraries
import os, time
os.environ['PYTHONHASHSEED']=str(1)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# %load_ext tensorboard
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use('seaborn-deep')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.titlesize'] = 14
plt.rcParams['figure.figsize'] = (12, 8)

pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 400)
import warnings
warnings.filterwarnings('ignore')
import sklearn.metrics as skm
import sklearn.model_selection as skms
import sklearn.preprocessing as skp
import random
seed = 12

<ipython-input-1-6d6a36f45a00>:11: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-deep')


In [2]:
import tensorflow as tf
from tensorflow import keras as k
print(tf.__version__)
from keras import backend as K
import os, shutil, re, string
import matplotlib.pyplot as plt
import spacy
seed=100

2.15.0


In [3]:
from skimage.transform import resize
from skimage import img_as_ubyte
from imageio import imread
import datetime

def set_seed():
    global seed
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed()

In [ ]:
import os
import zipfile

# Kaggle API 키를 설정합니다.
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Kaggle 데이터셋을 다운로드합니다.
!kaggle datasets download -d kmader/food41

# 다운로드한 데이터셋을 압축 해제합니다.
with zipfile.ZipFile("food41.zip", 'r') as zip_ref:
    zip_ref.extractall("food41")

mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/kmader/food41
License(s): copyright-authors
 86% 4.55G/5.30G [00:47<00:05, 137MB/s]

In [4]:
import json
folder_main = '/kaggle/input/food41'
folder_images = os.path.join(folder_main, 'images')
trainjson_fp = folder_main+'/meta/meta/train.json'
testjson_fp = folder_main+'/meta/meta/test.json'
with open(trainjson_fp, 'r') as f:
    trainjson = json.load(f)
with open(testjson_fp, 'r') as f:
    testjson = json.load(f)
labels = list(trainjson.keys())
labels[:5]

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/food41/meta/meta/train.json'